This notebook is used to develop functions for creating pointcloud labels from the cylinder models. Both the offset-vectors as well as the noise-labels are generated

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from fastprogress import progress_bar, master_bar
import os 
import sys

# Get access to all the files in the repository
cwd = os.getcwd()
parentDir = os.path.dirname( cwd )
sys.path.append(parentDir)

First load a single cylinder and test the function for finding the closest cylinder

In [24]:
cylinders = pd.read_csv( os.path.join(parentDir, 'data', 'raw', 'QSM', 'detailed', '33_1_000000.csv'), header=0)
print(cylinders.columns)
cylinders.columns = cylinders.columns.str.strip() # Clean whitespaces
print(cylinders.columns)
cylinders.head()

Index(['type', ' ID', ' parentID', ' startX', ' startY', ' startZ', ' endX',
       ' endY', ' endZ', ' radius', ' volume', ' growthVolume', ' length',
       ' growthLength', ' FittingType', 'segmentID', ' parentSegmentID',
       ' segmentMedianRadius', ' segmentGrowthVolume', ' segmentGrowthLength',
       ' branchOrder', ' reverseBranchOrder', ' reversePipeRadiusBranchorder',
       ' reversePipeAreaBranchorder', ' branchID', ' treeID', ' treeSpecies',
       ' translateX', ' translateY', ' translatez', ' gvA', ' gvB', ' gvC',
       ' glA', ' glB', ' glC'],
      dtype='object')
Index(['type', 'ID', 'parentID', 'startX', 'startY', 'startZ', 'endX', 'endY',
       'endZ', 'radius', 'volume', 'growthVolume', 'length', 'growthLength',
       'FittingType', 'segmentID', 'parentSegmentID', 'segmentMedianRadius',
       'segmentGrowthVolume', 'segmentGrowthLength', 'branchOrder',
       'reverseBranchOrder', 'reversePipeRadiusBranchorder',
       'reversePipeAreaBranchorder', 'branchID'

,type,ID,parentID,startX,startY,startZ,endX,endY,endZ,radius,...,treeSpecies,translateX,translateY,translatez,gvA,gvB,gvC,glA,glB,glC
0,cylinder,0,-1,27.268867,23.687374,-1.574510,27.310114,23.677517,-1.342370,0.147356,...,unknownSpecies,0.0,0.0,0.0,0.01,0.401606,0.0,0.01,0.401606,0.0
1,cylinder,1,0,27.310114,23.677517,-1.342370,27.323258,23.685699,-1.127219,0.147300,...,unknownSpecies,0.0,0.0,0.0,0.01,0.401606,0.0,0.01,0.401606,0.0
2,cylinder,2,1,27.323258,23.685699,-1.127219,27.372589,23.700213,-0.947553,0.147255,...,unknownSpecies,0.0,0.0,0.0,0.01,0.401606,0.0,0.01,0.401606,0.0
3,cylinder,3,2,27.372589,23.700213,-0.947553,27.363788,23.667894,-0.756015,0.147214,...,unknownSpecies,0.0,0.0,0.0,0.01,0.401606,0.0,0.01,0.401606,0.0
4,cylinder,4,3,27.363788,23.667894,-0.756015,27.380559,23.656645,-0.605488,0.147182,...,unknownSpecies,0.0,0.0,0.0,0.01,0.401606,0.0,0.01,0.401606,0.0


In [30]:
def closest_cylinder(point, cylinders):
    """
    Find the closest cylinder to a given point in 3D space.

    Parameters:
        point: A 3D point as a numpy array [x, y, z].
        cylinders: A list of dictionaries, where each dictionary represents a cylinder with:
                   - 'start': The start point of the cylinder axis as a numpy array [x, y, z].
                   - 'end': The end point of the cylinder axis as a numpy array [x, y, z].
                   - 'radius': The radius of the cylinder.

    Returns:
        The index of the closest cylinder and the corresponding distance.
    """
    closest_distance = float('inf')
    closest_index = -1
    closet_offset_vector = np.zeros(3)

    for _, cylinder in cylinders.iterrows():
        start = np.array([cylinder['startX'], cylinder['startY'], cylinder['startZ']])
        end = np.array([cylinder['endX'], cylinder['endY'], cylinder['endZ']])
        radius = cylinder['radius']
        id = cylinder['ID']

        # Cylinder axis vector
        axis = end - start
        axis_length = np.linalg.norm(axis)
        axis_unit = axis / axis_length

        # Vector from start point to the given point
        point_vector = point - start

        # Projection of point_vector onto the cylinder axis
        projection_length = np.dot(point_vector, axis_unit)
        projection_point = start + projection_length * axis_unit

        # Clamp the projection point to the cylinder segment
        projection_length_clamped = np.clip(projection_length, 0, axis_length)
        projection_point_clamped = start + projection_length_clamped * axis_unit

        # Compute the distance from the point to the clamped projection point
        distance_to_axis = np.linalg.norm(point - projection_point_clamped)

        # Subtract the cylinder radius to get the distance to the cylinder surface
        distance_to_cylinder_surface = abs(distance_to_axis - radius)

        # Update the closest cylinder
        if distance_to_cylinder_surface < closest_distance:
            closest_distance = distance_to_cylinder_surface
            closest_index = id
            closet_offset_vector = projection_point_clamped - point

    return closest_index, closest_distance, closet_offset_vector


In [31]:
# Example usage
point = np.array([26, 23, 20])

closest_idx, closest_dist, closet_offset_vector = closest_cylinder(point, cylinders)
print(f"The closest cylinder is at index {closest_idx} with a distance of {closest_dist} and vector {closet_offset_vector}")

The closest cylinder is at index 1111 with a distance of 0.7411319073719704 and vector [0.676347 0.217995 0.321168]


Now load an example pointcloud and perform the cylinder finding for every point

In [41]:
cloudPath = os.path.join( parentDir, "data", "raw", "cloud", "33_1.npy" )
cloud = np.load(cloudPath)

def generate_offset_cloud( cloud, cylinders, masterBar=None ):

    output_data = np.zeros((len(cloud), 7)) # point coordinates, offset vector, cylinder ID

    # for i, point in progress_bar(enumerate(cloud), total=len(cloud), master=masterBar):
    for i, point in enumerate(cloud):
        index, distance, offset_vector = closest_cylinder( point, cylinders )

        output_data[i] = np.concatenate( (point, offset_vector, [index]) )

        print(i)

    return output_data


In [42]:
labeled_cloud = generate_offset_cloud( cloud, cylinders )
print(labeled_cloud[:10,:])

0
1
2


KeyboardInterrupt: 